### Task 1. Form the following block and calculate its hash. To create a block you will have to initiate a new instance of the Block class and use its methods to calculate the hash.

In [1]:
d = {
    'index': 4,
    'timestamp': 0,
    'previous_hash':'000e69bd96b65c00653b4c59a5ece2f187578928460675c8317e8f91c74f8243',
    'nonce': 0,
    'author': 'Satoshi',
    'transactions': [
        {
            'sender_address': 'Alice',
            'recipient_address': 'Bob',
            'value': 10,
            'data': 'Hello Bob!',
        },
        {
            'sender_address': 'Bob',
            'recipient_address': 'Alice',
            'value':5,
            'data': 'Hello Alice!',
        }
    ]
}
pass

In [2]:
from blockchain import Transaction, Block

In [3]:
transaction1 = Transaction('Alice', 'Bob',10,'Hello Bob!')

In [4]:
transaction2 = Transaction('Bob', 'Alice',5,'Hello Alice!')

In [5]:
block = Block(d['index'],[transaction1,transaction2], d['author'], d['timestamp'],d['previous_hash'])

In [6]:
block.compute_hash()

'e8d96f7719c9fd6fece74ae1ba1012a27cad3c7d682786ac49025c39b11fb077'

### Task 2. Run PoW algorithm and find the proof for this block with difficulty=3. The PoW algorithm is run by the network peers. You will need to initiate an instance of a peer. Pay attention to the difficulty parameter.

In [7]:
from blockchain import BlockchainPeer

In [8]:
Peer = BlockchainPeer('Instance')

In [9]:
Peer.difficulty

3

In [10]:
Peer.proof_of_work(block)

'000f53dd44c6dd00a575e3f43b1a675812851a0df5d518555b026532f2772a32'

In [11]:
Peer.is_valid_proof(block,'000f53dd44c6dd00a575e3f43b1a675812851a0df5d518555b026532f2772a32')

True

### Task 3.  Init single peer and mine 2 blocks with the same 2 transactions from the task1. Why hashes for blocks (index = 1, 2) are not the same?

In [12]:
SinglePeer = BlockchainPeer('SP')

In [13]:
SinglePeer.add_new_transaction(transaction1)

In [14]:
SinglePeer.add_new_transaction(transaction2)

In [15]:
SinglePeer.mine()

In [16]:
SinglePeer.mine()

In [17]:
SinglePeer._get_chain()

{'length': 2,
 'chain': [{'index': 0, 'transactions': [], 'author': 'Satoshi', 'timestamp': 0, 'previous_hash': '0', 'nonce': 7099, 'hash': '000162ecd764ac73c5367af8c9fa659bbf2a2e57684c0657d219327de59bcf70'},
  {'index': 1, 'transactions': [{'sender_address': 'Alice', 'recipient_address': 'Bob', 'value': 10, 'data': 'Hello Bob!'}, {'sender_address': 'Bob', 'recipient_address': 'Alice', 'value': 5, 'data': 'Hello Alice!'}], 'author': 'SP', 'timestamp': 1695723892.5299315, 'previous_hash': '000162ecd764ac73c5367af8c9fa659bbf2a2e57684c0657d219327de59bcf70', 'nonce': 1285, 'hash': '000d35a1aa38296ffe36dac2be3244b214afb1befcf6794a93750299c25a4676'}]}

Хеши не одинаковые, потому что sha256 приняла новые параметры

### Task 4. Mine 100 blocks (in 1 block should be max 5 transactions) with 5 peers.  Keep in mind that you need to have 5 peers initiated, create a mainnet and launch the mining process. Moreover, the peers should have new transactions to add into blocks after the block is added into the mainnet.

### Which peer will have the most mined blocks with N>>100?

In [18]:
from blockchain import BlockchainMainnet

In [19]:
Peer1 = BlockchainPeer('1')
Peer2 = BlockchainPeer('2')
Peer3 = BlockchainPeer('3')
Peer4 = BlockchainPeer('4')
Peer5 = BlockchainPeer('5')

In [20]:
l = []
l.append(Peer1)
l.append(Peer2)
l.append(Peer3)
l.append(Peer4)
l.append(Peer5)

In [21]:
MainNet = BlockchainMainnet(l)

In [22]:
for i in range(100):
  for peer in l:
    peer.add_new_transaction(transaction1)
  #peer.add_new_transaction(transaction2)
  MainNet.run_mining()

In [23]:
MainNet.get_chain()

{'length': 101,
 'chain': [{'index': 0, 'transactions': [], 'author': 'Satoshi', 'timestamp': 0, 'previous_hash': '0', 'nonce': 7099, 'hash': '000162ecd764ac73c5367af8c9fa659bbf2a2e57684c0657d219327de59bcf70'},
  {'index': 1, 'transactions': [{'sender_address': 'Alice', 'recipient_address': 'Bob', 'value': 10, 'data': 'Hello Bob!'}], 'author': '2', 'timestamp': 1695723893.0413198, 'previous_hash': '000162ecd764ac73c5367af8c9fa659bbf2a2e57684c0657d219327de59bcf70', 'nonce': 12132, 'hash': '0004b34d78aa55a5a091d594c23ceae1ccaf683048e7b5ef3e614ec26c442ab5'},
  {'index': 2, 'transactions': [{'sender_address': 'Alice', 'recipient_address': 'Bob', 'value': 10, 'data': 'Hello Bob!'}], 'author': '5', 'timestamp': 1695723894.1119561, 'previous_hash': '0004b34d78aa55a5a091d594c23ceae1ccaf683048e7b5ef3e614ec26c442ab5', 'nonce': 9362, 'hash': '000003063afd1b6d53b624989765b442facd1969ee0538a5d39f02e6300b25fd'},
  {'index': 3, 'transactions': [{'sender_address': 'Alice', 'recipient_address': 'Bob', 

peer3 - 'author': '3' - 26
Перезапускал несколько раз, иногда другой пир имел максимальное значение. Поэтому при N >> 100 по ЗБЧ должно быть примерно одинаково у всех

### Task5. Suppose all blocks have the same timestamp as the genesis block. Mine 5 blocks with transactions from task1 - use the peer with the name 'Satoshi'. Use the template below to redefine the updated mine() function with fixed time.

### Find the hash of the block after the block from the task2.

In [24]:
from blockchain import BlockchainPeer, BlockchainMainnet
import time
from typing import List, Dict, Set, Optional
from hashlib import sha256
from copy import deepcopy

class FixedTimeBlockchainPeer(BlockchainPeer):
    # difficulty of our PoW algorithm
    difficulty = 3

    def __init__(self, peer_name: str):
        self.peer_name: str = peer_name
        self.unconfirmed_transactions: List[Transaction] = [] # mempool
        self.chain: List[Block] = None
        self.__init_blockchain()

    def create_genesis_block(self) -> None:
        """
        A function to generate genesis block and appends it to
        the chain. The block has index 0, previous_hash as 0, and
        a valid hash.
        """
        genesis_block = Block(
            index=0, transactions=[],
            author='Satoshi', timestamp=0,
            nonce=0, previous_hash="0",
        )
        genesis_block.hash = self.proof_of_work(genesis_block)
        self.chain = []
        self.chain.append(genesis_block)
        #logging.info(f"{self.peer_name} | Created genesis block {genesis_block.to_json()}")

    @property
    def last_block(self):
        return self.chain[-1]

    def __add_block(self, block: Block, proof: str):
        """
        A function that adds the block to the chain after verification.
        Verification includes:
        * Checking if the proof is valid.
        * The previous_hash referred in the block and the hash of latest block
          in the chain match.

        Args:
            block (Block): block to be added
            proof (str): proof of work result
        """
        #logging.info(f"{self.peer_name} | Adding block {block.to_json()}")

        previous_hash = self.last_block.hash

        if previous_hash != block.previous_hash:
            #logging.error(f"{self.peer_name} | Previous hash {previous_hash} != {block.previous_hash}")
            raise Exception("Invalid block")

        if not BlockchainPeer.is_valid_proof(block, proof):
            #logging.error(f"{self.peer_name} | Invalid proof {proof}")
            raise Exception("Invalid proof")

        block.hash = proof
        self.chain.append(block)
        #logging.info(f"{self.peer_name} | Added block {block}")

    @staticmethod
    def proof_of_work(block: Block) -> str:
        """
        Function that tries different values of nonce to get a hash
        that satisfies our difficulty criteria.

        Args:
            block (Block): block to be mined

        Returns:
            str: hash of the mined block
        """
        block.nonce = 0
        computed_hash = block.compute_hash()
        while not computed_hash.startswith('0' * BlockchainPeer.difficulty):
            block.nonce += 1
            computed_hash = block.compute_hash()
        return computed_hash

    def add_new_transaction(self, transaction: Transaction):
        self.unconfirmed_transactions.append(transaction)
        #logging.info(f"{self.peer_name} | Added transaction {transaction.to_json()}")

    def __init_blockchain(self):
        #logging.info(f"{self.peer_name} | Initializing blockchain")
        self.chain: List[Block] = []
        self.create_genesis_block()

    def _get_chain(self) -> Dict:
        """
        A function that returns the chain and its length.

        Returns:
            Dict: {
                'length': int - length of the chain,
                'chain': List[Block] - list of blocks in the chain
                'current_mainnet_peer_name': str - name of the current mainnet peer
                'peers': List[str] - list of peers names
            }
        """
        chain_data = []
        for block in self.chain:
            chain_data.append(block)

        return {
            "length": len(chain_data),
            "chain": chain_data,
        }

    def mine(self):
        """
        This function serves as an interface to add the pending
        transactions to the blockchain by adding them to the block
        and figuring out Proof Of Work.
        """
        #logging.info(f"{self.peer_name} | Start mining")

        #if not self.unconfirmed_transactions:
         #   logging.info(f"{self.peer_name} | No transactions to mine")
         #   return

        last_block: Block = self.last_block
        new_block = Block(
            index=last_block.index + 1,
            transactions=self.unconfirmed_transactions,
            author=self.peer_name,
            timestamp=0,
            previous_hash=last_block.hash,
            nonce=0,
        )
        proof = self.proof_of_work(new_block)
        self.__add_block(new_block, proof)
        self.unconfirmed_transactions = []
        self._announce()
    @classmethod
    def is_valid_proof(cls, block: Block, block_hash: str) -> bool:
        """
        Check if block_hash is valid hash of block and satisfies
        the difficulty criteria.

        Args:
            block (Block): block to be verified
            block_hash (str): hash of the block to be verified

        Returns:
            bool: True if block_hash is valid, False otherwise
        """
        return (block_hash.startswith('0' * BlockchainPeer.difficulty) and
                block_hash == block.compute_hash())

    @classmethod
    def check_chain_validity(cls, chain: List[Block]) -> bool:
        result = True
        previous_hash = "0"

        try:
            chain_copy = deepcopy(chain)
        except TypeError: # some attr is a couroutine
            return False

        for block in chain_copy:
            block_hash = block.hash
            # remove the hash field to recompute the hash again
            # using `compute_hash` method.
            delattr(block, "hash")

            if not cls.is_valid_proof(block, block_hash):
                #logging.error(f"Invalid proof {block_hash} for block {block.index} | valid proof {block.compute_hash()}")
                result = False
                break

            if previous_hash != block.previous_hash:
                #logging.error(f"Invalid previous hash {block.previous_hash} != {previous_hash}")
                result = False
                break

            block.hash, previous_hash = block_hash, block_hash

        return result

In [25]:
peers = [FixedTimeBlockchainPeer('Satoshi')]
mainnet = BlockchainMainnet(peers)

In [26]:
for i in range(5):
  for peer in peers:
    peer.add_new_transaction(transaction1)
    peer.add_new_transaction(transaction2)
  mainnet.run_mining()


In [27]:
mainnet.get_chain()

{'length': 6,
 'chain': [{'index': 0, 'transactions': [], 'author': 'Satoshi', 'timestamp': 0, 'previous_hash': '0', 'nonce': 7099, 'hash': '000162ecd764ac73c5367af8c9fa659bbf2a2e57684c0657d219327de59bcf70'},
  {'index': 1, 'transactions': [{'sender_address': 'Alice', 'recipient_address': 'Bob', 'value': 10, 'data': 'Hello Bob!'}, {'sender_address': 'Bob', 'recipient_address': 'Alice', 'value': 5, 'data': 'Hello Alice!'}], 'author': 'Satoshi', 'timestamp': 0, 'previous_hash': '000162ecd764ac73c5367af8c9fa659bbf2a2e57684c0657d219327de59bcf70', 'nonce': 698, 'hash': '00082515d5ffc84f2f8426fe56db666a1ad2adf807cdee15758dbcd2eb59b458'},
  {'index': 2, 'transactions': [{'sender_address': 'Alice', 'recipient_address': 'Bob', 'value': 10, 'data': 'Hello Bob!'}, {'sender_address': 'Bob', 'recipient_address': 'Alice', 'value': 5, 'data': 'Hello Alice!'}], 'author': 'Satoshi', 'timestamp': 0, 'previous_hash': '00082515d5ffc84f2f8426fe56db666a1ad2adf807cdee15758dbcd2eb59b458', 'nonce': 3180, 'hash